# 02 Stats Basics for ML

Correlation, collinearity, bias/variance, and overfitting basics.


## Table of Contents
- Correlation vs causation
- Multicollinearity (VIF)
- Bias/variance


## Why This Notebook Matters
Foundations notebooks build the intuition that prevents the most common mistakes in economic ML:
- leaking future information,
- evaluating with the wrong split strategy,
- over-interpreting coefficients.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Using random splits on time series.
- Assuming correlation implies causation.

## Matching Guide
- `docs/guides/00_foundations/02_stats_basics_for_ml.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/00_foundations/02_stats_basics_for_ml.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Concept
Core statistics ideas that show up constantly in ML: correlation, collinearity, bias/variance, and overfitting.


## Your Turn: Correlation vs Causation


In [ ]:
import numpy as np
import pandas as pd

# TODO: Simulate two correlated variables and a target
# Then compute correlations and explain why correlation != causation
...


## Your Turn: Multicollinearity (VIF)


In [ ]:
# TODO: Create two nearly identical features
# Compute VIF using src.econometrics.vif_table
...


## Your Turn: Bias/Variance


In [ ]:
# TODO: Fit a simple model vs a more flexible model on synthetic data
# Compare train vs test performance
...


## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Run a quick sanity check on any DataFrame/Series you created in this notebook.
# Example (adjust variable names):
# assert df.index.is_monotonic_increasing
# assert df.isna().sum().sum() == 0
#
# TODO: Write 2-3 sentences:
# - What would leakage look like in YOUR code?
# - How would you detect it?
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Correlation vs causation</summary>

```python
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
n = 800
z = rng.normal(size=n)
x = z + rng.normal(scale=0.8, size=n)
w = z + rng.normal(scale=0.8, size=n)
y = 2.0*z + rng.normal(scale=1.0, size=n)
df = pd.DataFrame({'x': x, 'w': w, 'y': y})
df.corr()
```

</details>

<details><summary>Solution: Multicollinearity (VIF)</summary>

```python
import statsmodels.api as sm
from src.econometrics import vif_table

# Make x2 highly correlated with x
df['x2'] = df['x'] * 0.95 + np.random.default_rng(1).normal(scale=0.2, size=len(df))
vif_table(df, ['x', 'x2'])

X = sm.add_constant(df[['x', 'x2']])
res = sm.OLS(df['y'], X).fit()
res.summary()
```

</details>

<details><summary>Solution: Bias/variance</summary>

```python
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

X = df[['x']].to_numpy()
y = df['y'].to_numpy()
split = int(len(df) * 0.8)

lin = LinearRegression().fit(X[:split], y[:split])
tree = DecisionTreeRegressor(random_state=0).fit(X[:split], y[:split])

rmse_lin_tr = mean_squared_error(y[:split], lin.predict(X[:split]), squared=False)
rmse_lin_te = mean_squared_error(y[split:], lin.predict(X[split:]), squared=False)
rmse_tree_tr = mean_squared_error(y[:split], tree.predict(X[:split]), squared=False)
rmse_tree_te = mean_squared_error(y[split:], tree.predict(X[split:]), squared=False)

(
    {'linear_train': rmse_lin_tr, 'linear_test': rmse_lin_te},
    {'tree_train': rmse_tree_tr, 'tree_test': rmse_tree_te},
)
```

</details>

